In [ ]:
def featureextraction(
    peptide_file_name: str,
    nucleotide_file_name: str,
    total: int
) -> list[list[float]]:
	import csv
	# import requests
	# import webbrowser
	# from selenium import webdriver
	# import re
	# import time
	import numpy as np

	# Final features
	feature: list[list[float]] = [[0 for x in range(872)] for y in range(total)]

	#amino acid feature set
	amino_acid: list[str] = [
		'A', 'R', 'N', 'D',
		'C', 'E', 'Q', 'G',
		'H', 'I', 'L', 'K',
		'M', 'F', 'P', 'S',
		'T', 'W', 'Y', 'V'
	]
     
	physicochemical_prroperties: dict[int, list[str]] = {
		0: ["Charged", "DEKHR"],
		1: ["Aliphatic", "ILV"],
		2: ["Aromatic", "FHWY"],
		3: ["Polar", "DERKQN"],
		4: ["Neutral", "AGHPSTY"],
		5: ["Hydrophobic", "CFILMVW"],
		6: ["Positive charged", "KRH"],
		7: ["Negative charged", "DE"],
		8: ["Tiny", "ACDGST"],
		9: ["Small", "EHILKMNPQV"],
		10:["Large", "FRWY"],
		11: ["Transmembrane amino acid", "ILVA"],
		12: ["dipole < 1.0 (A, G, V, I, L, F, P)", "AGVILFP"],
		13: ["1.0 < dipole < 2.0 (Y, M, T, S)", "YMTS"],
		14: ["2.0 < dipole < 3.0 (H, N, Q, W)", "HNQW"],
		15: ["dipole > 3.0", "RK"],
		16: ["dipole > 3.0 with opposite orientation (D, E)", "DE"]
	}

	#conjoint triad initialization
	S1: list[str] = ['A','G','V']
	S2: list[str] = ['I','L','F','P']
	S3: list[str] = ['Y','M','T','S']
	S4: list[str] = ['H','N','Q','W']
	S5: list[str] = ['R','K']
	S6: list[str] = ['D','E']
	S7: list[str] = ['C']

	N: np.ndarray = np.zeros((343), dtype=np.int32)
	feature1: np.ndarray = np.zeros((1, 343), dtype=np.int32)

	# digits, i, j, k - auxiliary variables for calculating N
	digits = np.arange(1, 8)
	i, j, k = np.meshgrid(digits, digits, digits, indexing='ij')
	N = (i * 100 + j * 10 + k).flatten().astype(np.int32)
    
	del digits

    # dipeptide initialization            
	dipeptide: list[str] = list()
	for i in range(len(amino_acid)):
		for j in range(0, len(amino_acid), 2):
			dipeptide.append(amino_acid[i] + amino_acid[j])
    
	str_ = ''
	line_number: int = 0
    
	print('Extracting features from amino-acid sequences...')
    
	with open(
		"E:/NeuralNetworks/bioinformatic_dataset/T6SE_training_data/T6SE_Training_Neg_1112.fasta",
		"r"
	) as handle:
		#single amino acid count
		for i in range(len(amino_acid)):
			feature[line_number][i] = round(
                number=str_.count(amino_acid[i]) / len(str_),
                ndigits=4
            )

		#dipeptide count    
		for i in range(len(dipeptide)):
			feature[line_number][i + 20] = round(
                number=str_.count(dipeptide[i]) / len(str_),
                ndigits=4
            )

		#physicochemical prroperties
		number: float = 0
		t: int = 865
		for key, value in physicochemical_prroperties.items():
			for j in value:
				number += str_.count(j)
			number /= len(str_)
			if key < 11:
				feature[line_number][i + 420] = round(
					number=number / len(str_),
					ndigits=4
				)
			else:
				feature[line_number][t] = round(
					number=number,
                    ndigits=4
				)
				t += 1
			


		for i in range(len(str_) - 2):
			p: int = 0
			X: str = str_[i]
			Y: str = str_[i + 1]
			Z: str = str_[i + 2]
			if X in S1:
				p = 100
			if X in S2:
				p = 200
			if X in S3:
				p = 3100
			if X in S4:
				p = 400
			if X in S5:
				p = 500
			if X in S6:
				p = 600
			if X in S7:
				p = 700   

			if Y in S1:
				p += 10
			if Y in S2:
				p += 20
			if Y in S3:
				p += 30
			if Y in S4:
				p += 40
			if Y in S5:
				p += 50
			if Y in S6:
				p += 60
			if Y in S7:
				p += 70

			if Z in S1:
				p += 1
			if Z in S2:
				p += 2
			if Z in S3:
				p += 3
			if Z in S4:
				p += 4
			if Z in S5:
				p += 5
			if Z in S6:
				p += 6
			if Z in S7:
				p += 7
		
			for j in range(343):
				if p == N[j]:
					k = j
			feature1[0, k] = feature1[0, k] + 1
		feature1[0, :] = feature1[0, :] / (len(str_) - 2) * 100
		feature[line_number][522:865] = feature1[0, :]
		feature1 = np.zeros((1, 343))
		str_=''
		
		line_number = line_number + 1
    
	for i in range(len(amino_acid)):
		feature[line_number][i] = str_.count(amino_acid[i]) / len(str_)
                
	for i in range(len(dipeptide)):
		feature[line_number][i + 20] = str_.count(dipeptide[i]) / len(str_)
                
    #physicochemical properties
    # Charged (DEKHR) 
    # Charged (DEKHR)
	number: float = 0
	t: int = 865
	for key, value in physicochemical_prroperties.items():
		for j in value:
			number += str_.count(j)
		number /= len(str_)
		if key < 11:
			feature[line_number][i + 420] = round(
				number=number / len(str_),
				ndigits=4
			)
		else:
			feature[line_number][t] = round(
				number=number,
				ndigits=4
			)
			t += 1

	for i in range(len(str_)-2):
		p: int = 0
		X: str = str_[i]
		Y: str = str_[i + 1]
		Z: str = str_[i + 2]
		if X in S1:
			p = 100
		if X in S2:
			p = 200
		if X in S3:
			p = 3100
		if X in S4:
			p = 400
		if X in S5:
			p = 500
		if X in S6:
			p = 600
		if X in S7:
			p = 700   

		if Y in S1:
			p += 10
		if Y in S2:
			p += 20
		if Y in S3:
			p += 30
		if Y in S4:
			p += 40
		if Y in S5:
			p += 50
		if Y in S6:
			p += 60
		if Y in S7:
			p += 70

		if Z in S1:
			p += 1
		if Z in S2:
			p += 2
		if Z in S3:
			p += 3
		if Z in S4:
			p += 4
		if Z in S5:
			p += 5
		if Z in S6:
			p += 6
		if Z in S7:
			p += 7
	
		for j in range(343):
			if p == N[j]:
				k = j
			feature1[0, k] = feature1[0, k] + 1
	feature1[0, :] = feature1[0, :] / (len(str_) - 2) * 100
	feature[line_number][522:865] = feature1[0, :]

	print('Amino acid feature extraction done!')
    #----------------------------------------------------------------------------------------
	print('Extracting features from nucleotide sequences...')
    #nucleotide feature set
	nucleotide: list[str] = ['A','T','G','C']
	dinucleotide: list[str] = list()
	trinucleotide: list[str] = list()

	for i in range(len(nucleotide)):
		for j in range(len(nucleotide)):
			dinucleotide.append(nucleotide[i] + nucleotide[j])
            
    
	for i in range(len(nucleotide)):
		for j in range(len(nucleotide)):
			for k in range(len(nucleotide)):
				trinucleotide.append(nucleotide[i]
						 			+ nucleotide[j]
									+ nucleotide[k])
             
	str_=''
	count=0
	line_number=0   
	with open(
		"E:/NeuralNetworks/bioinformatic_dataset/T6SE_training_data/T6SE_Training_Neg_1112.fasta",
		"r"
	) as handle:
		while line:
			if '>' not in line:
				str_=str_+line
			if '>' in line:
				#single nucleotide count
				for i in range(len(nucleotide)):
					feature[line_number][i+431]=round(str_.count(nucleotide[i])/len(str_),4)

				#dinucleotide count    
				for i in range(len(dinucleotide)):
					feature[line_number][i+4+431]=round(str_.count(dinucleotide[i])/len(str_),4)
				

				#trinucleotide count    
				for i in range(len(trinucleotide)):
					feature[line_number][i+4+16+431]=round(str_.count(trinucleotide[i])/len(str_),4)
				feature[line_number][871]=round((str_.count('G')+ str_.count('C'))/len(str_),4)
				line_number=line_number+1
				
			line=id.readline()
		
		#single nucleotide count
		for i in range(len(nucleotide)):
				feature[line_number][i+431]=round(str_.count(nucleotide[i])/len(str_),4)


		#dinucleotide count    
		for i in range(len(dinucleotide)):
				feature[line_number][i+4+431]=round(str_.count(dinucleotide[i])/len(str_),4)
				

		#trinucleotide count    
		for i in range(len(trinucleotide)):
				feature[line_number][i+4+16+431]=round(str_.count(trinucleotide[i])/len(str_),4)
		feature[line_number][871]=round((str_.count('G')+ str_.count('C'))/len(str_),4)         
    id.close()
    
    print('Nucleotide feature extraction done!')
    #-----------------------------------------------------------------------------------
    #secondary str_ucture
    print("Extracting secondary str_ucture and solvent accessibility features...")
    print("Please wait...")
    id=open(peptide_file_name,"r")
    line=id.readline()
    str_=''
    str_=str_+line
    line=id.readline()
    count=0
    line_number=0
    while line:
        if '>' not in line:
            str_=str_+line
        if '>' in line:
            count=count+1
            
            str_ucture=''
            solvent=''

            feature[line_number][515]=round((str_.count('E')+str_.count('A')+str_.count('L')+str_.count('M')+str_.count('Q')+str_.count('K')+str_.count('R')+str_.count('H'))/len(str_),4)
            feature[line_number][516]=round((str_.count('V')+str_.count('I')+str_.count('Y')+str_.count('C')+str_.count('W')+str_.count('F')+str_.count('T'))/len(str_),4)
            feature[line_number][517]=round((str_.count('G')+str_.count('N')+str_.count('P')+str_.count('S')+str_.count('D'))/len(str_),4)


            feature[line_number][518]=round((str_.count('A')+str_.count('L')+str_.count('F')+str_.count('C')+str_.count('G')+str_.count('I')+str_.count('V')+str_.count('W'))/len(str_),4)
            feature[line_number][519]=round((str_.count('R')+str_.count('K')+str_.count('Q')+str_.count('E')+str_.count('N')+str_.count('D'))/len(str_),4)
            feature[line_number][520]=round((str_.count('M')+str_.count('S')+str_.count('P')+str_.count('T')+str_.count('H')+str_.count('Y'))/len(str_),4)
            feature[line_number][521]=round((str_.count('M')+str_.count('S')+str_.count('P')+str_.count('T')+str_.count('H')+str_.count('Y'))/len(str_),4)
    
            line_number=line_number+1
            str_=''
            str_=str_+line
        line=id.readline()
        
    str_ucture=''
    solvent=''
    

    #secondary
    feature[line_number][515]=round((str_.count('E')+str_.count('A')+str_.count('L')+str_.count('M')+str_.count('Q')+str_.count('K')+str_.count('R')+str_.count('H'))/len(str_),4)
    feature[line_number][516]=round((str_.count('V')+str_.count('I')+str_.count('Y')+str_.count('C')+str_.count('W')+str_.count('F')+str_.count('T'))/len(str_),4)
    feature[line_number][517]=round((str_.count('G')+str_.count('N')+str_.count('P')+str_.count('S')+str_.count('D'))/len(str_),4)

    #solvent
    feature[line_number][518]=round((str_.count('Q')+str_.count('E')+str_.count('D'))/len(str_),4)
    feature[line_number][519]=round((str_.count('R')+str_.count('K')+str_.count('N')+str_.count('G'))/len(str_),4)
    feature[line_number][520]=round((str_.count('A')+str_.count('L')+str_.count('F')+str_.count('C')+str_.count('I')+str_.count('V'))/len(str_),4)
    feature[line_number][521]=round((str_.count('W')+str_.count('M')+str_.count('S')+str_.count('P')+str_.count('T')+str_.count('H')+str_.count('Y'))/len(str_),4)
                
    id.close()

    print('Extraction of secondary str_ucture and solvent accessibility features done!')
    with open("featurefile_subset.csv", 'w') as myfile:
      wr = csv.writer(myfile)
      wr.writerows(feature)
    return feature

In [ ]:
from Bio import SeqIO

with open(
    "bioinformatic_dataset/T6SE_training_data/T6SE_Training_Neg_1112.fasta",
    "r"
) as handle:
    i = 0
    for record in SeqIO.parse(handle, "fasta"):
        print(record.id)
        print(record.seq)
        # print(featureextraction("T6SE_Training_Neg_1112.fasta", "T6SE_Training_Neg_1112.fasta", 2))
        i += 1
        if i == 5:
            break

gi|56416452|ref|YP_153526.1|
MSLYYVRYWNTIKNDGRMVLMGKSRVAIYTDGACSGNPGPGGWGAVLRFGDGGERRISGGSDDTTNNRMELTAVIMALAALSGPCSVCVNTDSTYVKNGITEWIRKWKLNGWRTSNKSAVKNVDLWVELERLTLLHSIEWRWVKAHAGNEYNEEADMLARGEVERRMVIPK
gi|56416486|ref|YP_153560.1|
MVQISLPRNSKINPKGKVHNMAEGAQRVGCFKIYRWSPDDDEGPRIDTFYIDLDKCGQMVLDALIKIKNEVDSTLTFRRSCREGICGSCAMNIDGSNTLACTKYISDIKGDVKIYPLPHMNVIKDLVPDLSDFYAQYKSVSPWLKAKDPVSGTSERLQSVEDRSKLDGIYDCILCASCSTSCPSYWWNSDKYLGPAVLLQVYRWLADSRDEATDERLELLDDAFKLYRCHTIMNCTKTCPKDLNPAKAISKIKQLMLKRLEL
gi|56416658|ref|YP_153732.1|
MCALLVLRFGYMSSLGVNKVILIGHLGKDPDVRVMQNGKEMASFSLATSESWLDKASGARTEKTEWHSIVVFSEGLVRVVKTCARKGSKVYIEGSLRTRKWSDQNGNDRYTTEVVLQGFNSALCLLDPRGGGAATQDALQNAAGPHEEPGMCVDDAGLPENAIVEEVSFADEDMDEIPF
gi|56416728|ref|YP_153802.1|
MVSLDLLPDALNMRAASFFDRCVHALSCSGLFASDMAIDLGTANTLVYVKGRGIVLNEPSVVAILKERGNYVPYAFGAKAKMMLGKTPGGIEAIRPLKDGVIADFKGAEEMIKFFIRNASKRRTINSPTVVICVPSGSTPVERRAIQDAAEGAGAGEVFLIEEPMAAAIGAGMPVTQPEASMIVDVGGGTTEVAVISLGGIVYSRSVRSGGDLMDEAIASYIRKHYNLLVGETTAERIKKSIGAACPPESGDGDTVIVKGRDLSDGVPK

In [24]:
from Bio import SeqIO
from collections import Counter
import numpy as np

def calculate_aac(sequence: str) -> dict:
    """Calculate Amino Acid Composition"""
    aac = Counter(sequence)
    total_length = len(sequence)
    
    # Normalize to frequencies
    return {aa: count/total_length for aa, count in aac.items()}

def calculate_dpc(sequence: str) -> dict:
    """Calculate Dipeptide Composition"""
    dpc = {}
    total_pairs = len(sequence) - 1
    for i in range(len(sequence) - 1):
        dipeptide = sequence[i:i+2]
        dpc[dipeptide] = dpc.get(dipeptide, 0) + 1
    
    # Normalize to frequencies
    return {dp: count/total_pairs for dp, count in dpc.items()}

def calculate_qso(sequence, max_lag=30, weight=0.1):
    """Calculate Quasi-sequence-order descriptors"""
    n = len(sequence)
    # Calculate sequence order descriptors
    seq_order_descriptors = []
    for lag in range(1, max_lag + 1):
        if lag < n:
            descriptor = 0
            for j in range(n - lag):
                # Simple example: difference based on amino acid type
                # In practice, you would use a similarity matrix
                if sequence[j] != sequence[j + lag]:
                    descriptor += 1
            seq_order_descriptors.append(descriptor / (n - lag))
    
    # Combine with AAC using weighting factor
    aac = calculate_aac(sequence)
    # Normalization factor
    return {"aac": aac, "sequence_order": seq_order_descriptors}

for record in SeqIO.parse(
    "bioinformatic_dataset/T6SE_training_data/T6SE_Training_Neg_1112.fasta",
    "fasta"
):
    aac_features = calculate_qso(str(record.seq))
    print(f"AAC for {record.id}: {aac_features}")

AAC for gi|56416452|ref|YP_153526.1|: {'aac': {'M': 0.04093567251461988, 'S': 0.06432748538011696, 'L': 0.07602339181286549, 'Y': 0.03508771929824561, 'V': 0.08187134502923976, 'R': 0.08187134502923976, 'W': 0.04678362573099415, 'N': 0.07017543859649122, 'T': 0.05847953216374269, 'I': 0.04678362573099415, 'K': 0.05263157894736842, 'D': 0.04678362573099415, 'G': 0.10526315789473684, 'A': 0.07017543859649122, 'C': 0.017543859649122806, 'P': 0.023391812865497075, 'F': 0.005847953216374269, 'E': 0.06432748538011696, 'H': 0.011695906432748537}, 'sequence_order': [0.9294117647058824, 0.9289940828402367, 0.8988095238095238, 0.9281437125748503, 0.963855421686747, 0.9393939393939394, 0.9146341463414634, 0.9325153374233128, 0.9320987654320988, 0.9130434782608695, 0.93125, 0.9371069182389937, 0.9556962025316456, 0.9490445859872612, 0.9743589743589743, 0.9354838709677419, 0.9675324675324676, 0.9477124183006536, 0.9473684210526315, 0.8940397350993378, 0.9733333333333334, 0.9530201342281879, 0.89864